# Step 1: Data Collection

## 1. Collecting GYEN Data:

In [ ]:
import ccxt

# List of exchanges to try
exchanges = ['kraken', 'huobi', 'bitfinex', 'coinbase']

for exchange_name in exchanges:
    try:
        exchange = getattr(ccxt, exchange_name)()
        markets = exchange.load_markets()
        for market in markets:
            if 'GYEN' in market:
                print(f'{market} found on {exchange_name}')
    except Exception as e:
        print(f"Error with {exchange_name}: {e}")

In [ ]:
import ccxt
import pandas as pd

# Initialize the Coinbase API
exchange = ccxt.coinbase()

# Set the start date to March 12, 2021, at 9 AM (UTC)
start_date = '2021-03-12T09:00:00Z'

# Fetch GYEN/USD data from Coinbase
ohlcv = exchange.fetch_ohlcv('GYEN/USD', timeframe='1d', since=exchange.parse8601(start_date))

# Convert to DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
gyen_usd_df = pd.DataFrame(ohlcv, columns=columns)

# Convert timestamp to datetime
gyen_usd_df['timestamp'] = pd.to_datetime(gyen_usd_df['timestamp'], unit='ms')

# Display the first few rows
print(gyen_usd_df.head())

# Save to CSV if needed
gyen_usd_df.to_csv('GYEN_USD_data_coinbase.csv')